In [1]:
import time
import random
import requests
import pandas as pd
import urllib.request
from urllib.parse import quote

import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver  # 동적크롤링
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
# review_scrapy_data : 전처리가 끝난 visitJeju의 API 전체 CSV
data = pd.read_csv("Data/visit_jeju_scrapy_data.csv", index_col=False)

In [3]:
data.head()

# 여기서 이용할 컬럼은 contentsid 하나.

,title,category,introduction,address,roadaddress,latitude,longitude,phoneno,imgpath,tag,contentsid
0,미유,쇼핑,숲속 별장처럼 꾸며진 소품 가게,제주특별자치도 제주시 한림읍 옹포리 326-3,제주특별자치도 제주시 한림읍 한림상로 15-5,33.405636,126.256762,0507-1349-9322,https://api.cdn.visitjeju.net/photomng/imgpath...,"핸드메이드소품, 키링, 우산, 한림, 옹포리, 소품샵, 지갑, 쇼핑",CNTS_200000000015086
1,마야블루,쇼핑,마야블루는 제주 시내의 주택가 사이에 작은 간판으로 자신의 존재를 알리고 있다. 이...,제주특별자치도 제주시 노형동 1052-27,제주특별자치도 제주시 월랑로6길 21,33.489570,126.478593,010-8515-2470,https://api.cdn.visitjeju.net/photomng/imgpath...,"악세사리, 쇼핑,라탄,원데이클래스,관광기념품,상점/상가, 쇼핑,라탄,원데이클래스,카...",CNTS_200000000007334
2,몸냥공작소,쇼핑,귀엽고 제주스러운 유니크한 소품가게,제주특별자치도 제주시 애월읍 유수암리 2503-1,제주특별자치도 제주시 애월읍 하소로 595,33.430614,126.397470,--,https://api.cdn.visitjeju.net/photomng/imgpath...,"공방,기념품,,아주 어려움, 공방,기념품,쇼핑,관광기념품,상점/상가",CNTS_000000000022837
3,은인마켙,쇼핑,"자개, 유리 등 다양한 식기류를 만나볼 수 있는 소품샵",제주특별자치도 제주시 조천읍 함덕리 1082,제주특별자치도 제주시 조천읍 함덕서2길 20,33.540661,126.663099,010-3006-7721,https://api.cdn.visitjeju.net/photomng/imgpath...,"소품샵, 함덕, 잡화, 식기",CNTS_200000000015021
4,뱅뱅와인마켓,쇼핑,공항 인근에 위치한 대형 와인판매점,제주특별자치도 제주시 오라이동 2171-1,제주특별자치도 제주시 사평2길 9,33.492220,126.510559,064-746-4141,https://api.cdn.visitjeju.net/photomng/imgpath...,"샴페인, 오라동, 와인, 제주시내",CNTS_200000000014989


In [4]:
data.isna().sum()

title           0
category        0
introduction    0
address         0
roadaddress     0
latitude        0
longitude       0
phoneno         0
imgpath         2
tag             0
contentsid      0
dtype: int64

# contentsid를 이용해 검색하여 해당 이름을 불러올 예정

In [5]:
df = pd.DataFrame(data)

In [6]:
df_cocntentsid = df[['title','contentsid']]
df_cocntentsid['contentsid'][0]

'CNTS_200000000015086'

In [7]:
df_cocntentsid['title'][0]

'미유'

> contentsid를 통해 검색을 할것. 그러나 title이 뭔지는 알아야하니깐

# 검색할 항목 정리
- https://www.visitjeju.net/kr/detail/view?contentsid=
- df_contentsid['contentsid'][index]

# 어떻게 추가할까?
- df_contentsid 데이터프레임에 새로운 컬럼을 만들어 붙디던가,
- 새로운 데이터 프레임을 만들어 크롤링한 데이터를 저장하고 df_contentsid랑 합치던가.


In [8]:
df_cocntentsid['contentsid'][2000]

'CNTS_000000000019679'

# 크롤링 할 데이터 정리
- class "inner_wrap" 데이터(상단 데이터) - 상단 이미지 데이터의 경우 이미 있는 imgPath와 동일하기 때문에 필요 없음.
    - title "//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[1]/h3"
    - 별점 "//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[3]/p"
    - mainTag(해당 관광지를 분류 가능) 
    "//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[4]/p[1]"
        //*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[4]/p[1]/a[1]
        //*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[4]/p[1]/a[2]
        //*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[4]/p[1]/a[3]
    - subTag(해당 관광지를 조금더 디테일하게 설명 )
    "//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[4]/p[2]"
        //*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[4]/p[2]/a[1]
        .
        .
        .
        //*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[4]/p[2]/a[7]

- class "add2020_detail_left" 데이터(본문내용 좌측데이터 = 상세정보)
    - image 규칙이 없음. 있긴한데 div[3]에는 text가 있고 1,2,4,5에는 이미지가 있고, 
        - //*[@id="tab0"]/div/div[1]/div[1]/div/div/div/div/img
        - //*[@id="tab0"]/div/div[1]/div[2]/div/div/div/div[1]/img
        - //*[@id="tab0"]/div/div[1]/div[2]/div/div/div/div[2]/img
        - //*[@id="tab0"]/div/div[1]/div[4]/div/div/div/div/img
        - //*[@id="tab0"]/div/div[1]/div[5]/div/div/div/div/img
    - text
        - 와 텍스트가 html언어네 이거 어케 긁어오지? 한번에 긁어올 수 있는 방법 필요.
        //*[@id="tab0"]/div/div[1]/div[3]/div/div/div

- class "add2020_detail_right" 중 이용안내 파트만 긁어오기. 왜냐하면 

# 크롬 켜기

In [36]:
# Chrome Browser 와 Chrome Driver Version 확인하기
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),options = chrome_options)
driver.get(f"https://www.visitjeju.net/kr/detail/view?contentsid={df_cocntentsid['contentsid'][0]}")

# 데이터 수집 테스트

In [ ]:
# # 연습코드
# import requests
# from lxml import html

# # 크롤링할 웹페이지 URL 설정
# url = 'https://www.visitjeju.net/kr/detail/view?contentsid=CNTS_000000000019279'
# driver.get(url)
# scrapy_data = pd.DataFrame(columns=['text1', 'image1'])

# text1 = []
# image1 = []



# try:
#     soup = BeautifulSoup(html, 'html.parser')
#     dl_tag = soup.find('dl')
#     # 텍스트 가져오기
#     text_element = soup.find('div', class_='wrap_contView')
#     text = text_element.get_text(strip=True)
    
#     # 이미지 가져오기
#     image_element = soup.find('div', class_='swiper-slide')
#     image_url = image_element.find('img')['src']
    
#     # 데이터프레임에 저장
#     data = {'text1': [text], 'image1': [image_url]}
#     scrapy_data = pd.DataFrame(data)

# except Exception as e:
#     print(e)

In [ ]:
# text1

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# # 크롤링할 웹페이지 URL 설정
# url = 'https://www.visitjeju.net/kr/detail/view?contentsid=CNTS_000000000019279'

# # 웹페이지에 GET 요청 보내고 HTML 가져오기
# response = requests.get(url)
# html = response.text

# try:
#     soup = BeautifulSoup(html, 'html.parser')
    
#     # dl 태그 찾기
#     dl_tag = soup.find('dl')
    
#     if dl_tag:
#         # 텍스트와 이미지를 저장할 변수 초기화
#         text1 = ""
#         image1 = None
        
#         # dt와 dd 태그를 모두 찾아서 반복문으로 순회
#         for dt_tag, dd_tag in zip(dl_tag.find_all('dt'), dl_tag.find_all('dd')):
#             # dt 태그의 텍스트 추출 (제목)
#             dt_text = dt_tag.get_text(strip=True)
#             # dd 태그의 텍스트 추출 (상세 내용)
#             dd_text = dd_tag.get_text(strip=True)
            
#             # 상세 내용에 이미지가 있는지 확인
#             img_tag = dd_tag.find('img')
#             if img_tag:
#                 # 이미지의 src 속성 값 추출
#                 image1 = img_tag.get('src')
            
#             # dt 태그의 텍스트와 dd 태그의 텍스트를 합쳐서 저장
#             if dt_text:
#                 text1 += f"{dt_text}: {dd_text}\n"
        
#         # 데이터프레임에 저장
#         data = {'text1': [text1], 'image1': [image1]}
#         scrapy_data = pd.DataFrame(data)
#     else:
#         print("dl 태그를 찾을 수 없습니다.")
# except Exception as e:
#     print(e)


In [ ]:
# data

In [ ]:
# scrapy_data

In [ ]:
# # 크롤링할 웹페이지 URL 설정
# url = 'https://www.visitjeju.net/kr/detail/view?contentsid=CNTS_000000000019279'
# driver.get(url)

# soup = BeautifulSoup(html, 'html.parser')
    
# # # dl 태그 찾기
# # XPath로 dl 태그 찾기
# dl_element = driver.find_element(By.XPATH,"/html/body/div/div[2]/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div[2]/div[2]/div")
# try:
#     # dt = title
#     dt_elements = dl_element.find_elements(By.TAG_NAME, "dt")
#     # dd = detail
#     dd_elements = dl_element.find_elements(By.TAG_NAME, "dd")

#     for dt, dd in zip(dt_elements, dd_elements):
#         print(f"{dt.text}-{dd.text}")
    

# except Exception as e:
#     print(e)
#     print('크롤링 실패')
    
    


In [ ]:
# dd_elements


In [ ]:
# //*[@id="content"]/div[2]/div[2]/div[2]/div[2]/div[2]/div
# //*[@id="content"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/dl/dt[1] #소개
# //*[@id="content"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/dl/dd[1]/text() #관광지와 가까운 마니주펜션
# //*[@id="content"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/dl/dt[2] #이용시간
# //*[@id="content"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/dl/dd[2]/text()

# //*[@id="content"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/dl/dd[3] #공용주차장...

In [ ]:
# # 크롤링할 웹페이지 URL 설정
# url = 'https://www.visitjeju.net/kr/detail/view?contentsid=CNTS_000000000019279'
# driver.get(url)

# soup = BeautifulSoup(html, 'html.parser')
    
# # # dl 태그 찾기
# # XPath로 dl 태그 찾기
# # test_element = driver.find_element(By.XPATH, "/html/body/div/div[2]/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div[2]/div[2]")
# dl_element = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div[2]/div[2]/div')
# try:
#     # dt = title
#     dt_elements_save = []
#     # dt_elements = dl_element.find_elements(By.TAG_NAME, "//dl/dt")
#     dt_elements = dl_element.find_elements(By.XPATH, "/dl/dt")
#     # dd = detail
#     dd_elements_save = []
#     dd_elements = dl_element.find_elements(By.XPATH, "/dl/dd")

#     for idx, dt, dd in zip(dt_elements, dd_elements):
#         idx + 1
#         dt_elements_save.append(dt[idx])
#         dd_elements_save.append(dd[idx])
#         print(f"{dt.text}-{dd.text}")
    

# except Exception as e:
#     print(e)
#     print('크롤링 실패')
    
    


# body Left 부분

In [ ]:
from selenium.webdriver.common.by import By

# 요소들을 가져올 XPath
xpath = '/html/body/div/div[2]/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div[1]/div[1]/div/div[1]//*'

# 이미지와 텍스트를 모두 가져오는 함수
def get_images_and_texts(driver, xpath):
    elements = driver.find_elements(By.XPATH, xpath)
    images = []
    texts = []

    for element in elements:
        if element.tag_name == 'img':
            images.append(element.get_attribute('src'))
        else:
            # p 태그 안에 있는 text만 가져오기
            p_elements = element.find_elements(By.XPATH, './/p')
            for p_element in p_elements:
                text = p_element.text.strip()
                if text:
                    texts.append(text)
    # 중복된 항목 제거
    unique_texts = list(set(texts))
    return images, unique_texts

# 이미지와 텍스트 가져오기
images, texts = get_images_and_texts(driver, xpath)

# 결과 출력
for image in images:
    print("Image Link:", image)

for text in texts:
    print("Text:", text)


Image Link: https://api.cdn.visitjeju.net/photomng/imgpath/202306/30/fde82514-8196-4b6e-8d4a-4f6620b22015.jpg
Image Link: https://api.cdn.visitjeju.net/photomng/imgpath/202306/30/3df8d74d-20f1-4e46-8fe6-0485f068addf.jpg
Image Link: https://api.cdn.visitjeju.net/photomng/imgpath/202306/30/75784c15-6e95-4b54-9b59-c81a20b5b566.jpg
Image Link: https://api.cdn.visitjeju.net/photomng/imgpath/202306/30/9e8c8477-f45b-443f-a6d6-f81427d9a861.jpg
Image Link: https://api.cdn.visitjeju.net/photomng/imgpath/202306/30/3008c187-5d8b-4f99-b51e-f1e03d92a70c.jpg
Image Link: https://api.cdn.visitjeju.net/photomng/imgpath/202306/30/e1084722-894d-49d6-ab94-de849ca492d9.jpg
Text: 호끄만거는 관광객들로 붐비는 제주 동문시장 청년몰 내에 위치한 인테리어 소품샵이며 동문시장 공영주차장 지하 1층으로 찾아가면 된다.
Text: 직접 촬영한 제주의 아름다운 풍경을 담은 엽서를 비롯해 제주에서 활발히 활동하고 있는 작가들이 만든 소품들을 판매한다. 감귤초콜릿, 감귤 모자, 한라산 기념품으로 가득 찬 관광기념품 샵과는 달리 다른 곳에서는 판매하지 않는 감각 있고 특별한 소품들이 있다. 인센스 홀더와 명함이나 인쇄물을 꽂을 수 있는 각종 홀더는 한라산, 바람, 구름 등 제주의 자연에서 영감을 받아 돌로 제작한 소품으로 특별한 기념품을 찾는 관광객에게 인기가 많은 아이템이다.
Tex

# 본 코드

In [74]:
# Chrome Browser 와 Chrome Driver Version 확인하기
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),options = chrome_options)
driver.get(f"https://www.visitjeju.net/kr/detail/view?contentsid={df_cocntentsid['contentsid'][0]}")

In [73]:
# 본 코드
mainUrl= "https://www.visitjeju.net/kr/detail/view?contentsid="
contentsid = df_cocntentsid['contentsid']
# 크롤링 설정
idx = 0 # 크롤링할 데이터 총 갯수
failed_idx = 0 # 크롤링 실패시 1 증가

t = random.randrange(1,2)

# 크롤링할 데이터 담길 곳(header, body_left, body_right)
    # header (title, stars, mainTag, subTag)
scrapy_header = pd.DataFrame(columns=['title', 'stars', 'mainTag', 'subTag'])
    # body_left (additional_image, mainText)
scrapy_left = pd.DataFrame(columns=['BodyLeftImage', 'BodyLeftText'])
    # body_right (subInformTitle == dt, subInformText == dd)
scrapy_right = pd.DataFrame(columns=['BodyRightText'])


# 본문 크롤링
for index, contentName in enumerate(contentsid):
    print(f"=================현재 page : {index}, title : {df_cocntentsid['title'][index]}====================")
    time.sleep(t)
    driver.get(f"{mainUrl}{contentsid[index]}")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 추천 버튼 뜨면 클릭
    try:
        # 추천 버튼이 있는지 확인
        recommend_button = driver.find_element(By.XPATH, '//*[@id="footer"]/div[4]/button')
        
        # 추천 버튼이 있을 경우 클릭
        if recommend_button.is_displayed():
            recommend_button.click()
            print("===================버튼클릭완료 ===================")
            time.sleep(t)
    except Exception as e:
        print("추천 버튼이 없습니다.")

    
    # ========================header 크롤링=======================
    # title, stars, mainTag, subTag
    headerSelector = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div[2]/div[2]/div/div/div[2]/div[1]/div[1]')
    try:
        header_title = headerSelector.find_elements(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[1]/h3')
        header_stars = headerSelector.find_elements(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[3]/p')
        header_mainTag = headerSelector.find_elements(By.CLASS_NAME, "best_tag")
        # header_subTag = headerSelector.find_elements(By.XPATH, '//p[@data-v-51160e04]')
        header_subTag = headerSelector.find_elements(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[4]/p[2]')
        # title, stars
        for h_title, h_stars in zip(header_title, header_stars):
            print(h_title.text,"-", h_stars.text)

        # mainTag
        for midx, h_mainTag in enumerate(header_mainTag):
            # 태그 분리
            tags = h_mainTag.text.split("#")
            tags = [f"main태그{index}: #{tag.strip()}" for tag in tags if tag.strip()]  # "#"을 다시 추가하여 태그 리스트 작성
            print(", ".join(tags))  # 분리된 태그들을 출력
        # subTag
        for midx, h_subTag in enumerate(header_subTag):
            stags = h_subTag.find_elements(By.TAG_NAME, 'a')
            for stag in stags:
                print(f"sub태그{index} : {stag.text}")
    
    except Exception as e:
        print(e)
        print("헤더 크롤링 실패")
        continue
    
    time.sleep(t)
    # ========================bodyLeft크롤링=======================
    # bodyLeftText, bodyLeftImage
        # image 링크 예시
        # api.cdn.visitjeju.net/photomng/imgpath/202306/15/05dc24f9-60b7-415b-bf27-57361e59147d.jpg
    bodyLeftSelector = '/html/body/div/div[2]/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div[1]/div[1]/div/div[1]//*'
    bodyLeftImages = []
    bodyLeftTexts = []
    # 이미지와 텍스트를 모두 가져오는 함수
    def get_images_and_texts(driver, bodyLeftSelector):
        elements = driver.find_elements(By.XPATH, bodyLeftSelector)

        for element in elements:
            if element.tag_name == 'img':
                bodyLeftImages.append(element.get_attribute('src'))
            else:
                # p 태그 안에 있는 text만 가져오기
                p_elements = element.find_elements(By.XPATH, './/p')
                for p_element in p_elements:
                    text = p_element.text.strip()
                    if text:
                        bodyLeftTexts.append(text)
        # 중복된 항목 제거
        unique_texts = list(set(bodyLeftTexts))
        return bodyLeftImages, unique_texts

    # 이미지와 텍스트 가져오기
    try:
        bodyLeftImages, unique_texts = get_images_and_texts(driver, bodyLeftSelector)
        # 결과 출력
        for idx, image in enumerate(bodyLeftImages):
            print(f"{idx}BodyLeftImage:", image)

        for idx, text in enumerate(unique_texts):
            print(f"{idx}BodyLeftText:", text)

        # 데이터프레임에 저장
        scrapy_left = scrapy_left.append({'BodyLeftImage': ','.join(bodyLeftImages), 'BodyLeftText': ','.join(unique_texts)}, ignore_index=True)
        
    except Exception as e:
        print(e)
        print("bodyLeft크롤링 실패")
        continue



    # ========================bodyRight 크롤링=======================
    bodyRightData = [] # 상세정보
    # dl 태그 찾기
    dl_element = driver.find_element(By.XPATH,"/html/body/div/div[2]/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div[2]/div[2]/div")
    try:
        # dt = title
        dt_elements = dl_element.find_elements(By.TAG_NAME, "dt")
        # dd = detail
        dd_elements = dl_element.find_elements(By.TAG_NAME, "dd")
        time.sleep(1)
        # dt_elements와 dd_elements의 text 출력
        for dt, dd in zip(dt_elements, dd_elements):
            print(f'bodyRightText : {dt.text}-{dd.text}')
            bodyRightData.append(f"{dt.text}-{dd.text}")

    except Exception as e:
        print(e)
        print('bodyRight크롤링 실패')



=================현재 page : 0, title : 미유====================
추천 버튼이 없습니다.
미유 - 별점(별점없음)
main태그0: #한림, main태그0: #옹포리, main태그0: #소품샵
sub태그0 : #핸드메이드소품
sub태그0 : #키링
sub태그0 : #우산
sub태그0 : #지갑
sub태그0 : #쇼핑
0BodyLeftImage: https://api.cdn.visitjeju.net/photomng/imgpath/202306/13/34c0934e-f750-49db-9a3a-7382581d8924.jpg
1BodyLeftImage: https://api.cdn.visitjeju.net/photomng/imgpath/202306/13/05c35d7a-6be0-4e81-b93f-1919dce0b077.jpg
2BodyLeftImage: https://api.cdn.visitjeju.net/photomng/imgpath/202306/13/90e8ed4a-91ae-4506-81ac-f67914d0fc63.jpg
3BodyLeftImage: https://api.cdn.visitjeju.net/photomng/imgpath/202306/13/e97ea3a8-790a-4041-9953-c06f1f19ec29.jpg
4BodyLeftImage: https://api.cdn.visitjeju.net/photomng/imgpath/202306/13/4f0a2f93-6e0a-47ca-a710-6ab9ff1168b1.jpg
5BodyLeftImage: https://api.cdn.visitjeju.net/photomng/imgpath/202306/13/0c7e5875-c0d5-4f63-96b6-0176dd4ce854.jpg
0BodyLeftText: 가게 규모가 크진 않지만, 제주 여행 기념품 종류를 다양하게 구비하고 있어 둘러보는 재미가 쏠쏠한 편. 귤 모양 파우치, 스트링 백과 같은 사장님이 직접 만든 핸드메이드 제품도 선